# CSI4142 - Group 48 - Assignment 3 - Part 1

---

## Introduction
In this report, we will conduct an empirical study to evaluate a linear regression approach on a regression task. This will be conducted on the ___ daatset. For this study, we will follow the following steps:

1. Clean the data 
2. Encode categorical features to transform them into numerical features 
3. Conduct an EDA (Exploritory Data Analysis) to visualize data and find outliers in the features using LOF (Local Outlier Factor)
4. (Optional) Explore the LinearRegression method suggested in scikit-learn (or other packages)
5. Program a feature aggregator to create 2 additional features
6. Split the data into train, validation, and test sets, choose an evaluation metric (e.g., MSE, RMSE, R²), establish a baseline using linear regression without outlier removal or feature aggregation, perform 4-fold cross-validation on different system variations, select the best model, and evaluate it on the untouched test set.
7. Analyize the results
8. Discuss the outliers and feature aggregation, as well as the results on the unseen test set compare to the cross-validation results


#### Group 48 Members
- Ali Bhangu - 300234254
- Justin Wang - 300234186

<br>

---